## Objective
Integration of LLM with Vector DB using Langchain

In [180]:
from langchain.chains import RetrievalQA, QAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import qdrant
from qdrant_client import QdrantClient
from langchain import OpenAI
from qdrant_client.http import models
from langchain.llms import HuggingFaceTextGenInference
embeddings_model_name = "all-MiniLM-L6-v2"

# Connecting to TGI model server

In [181]:
tgi_llm = HuggingFaceTextGenInference(
    inference_server_url="http://34.143.224.157:8000/",
    max_new_tokens=512,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.8,
    repetition_penalty=1.03,
)

## Qdrant server

In [182]:
client = QdrantClient(url="http://34.173.165.171",port=6333)
collection_name = "VideoCollection"
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)
qdrant_instance = qdrant.Qdrant(client, collection_name, embeddings=embeddings)

### Query parameters

In [183]:
query = "What are the skincare and haircare products discussed by James Welsh?"
creater_filter = 'James Welsh'

In [184]:
retriever = qdrant_instance.as_retriever(search_kwargs={"k": 35
,"filter": models.Filter(
    must=[models.FieldCondition(key="metadata.creatorName", match=models.MatchAny(any=[creater_filter]))
        #  ,models.FieldCondition(key ="metadata.brandsMentioned", match=models.MatchAny(any=['nespresso']))
         ]
      )})

docs = retriever.get_relevant_documents(query)
# Giving sources in documents
for i,d in enumerate(docs):
    d.metadata['source'] = i

#### QA chain of Langchain + LLM  ---- QA with sources.

In [ ]:
query_dic = {'question':query,
            'docs':  docs}
# print(query_dic)
qa_src = QAWithSourcesChain.from_chain_type(llm=tgi_llm, chain_type="stuff", return_source_documents = True)
res = qa_src(query_dic)
print("\n------------Results--------------\n",res)

### RetrievalQA

In [186]:
qa = RetrievalQA.from_chain_type(llm=tgi_llm, chain_type="stuff", retriever=retriever)
result=qa(query)
print("\n\nResults from retrievalQA------->\n\n",result)


Prompts-----
 [StringPromptValue(text="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nskincare.\n\ngathering loads of information from real skincare experts, I've come up with a few\n\nmany ingredients for a plethora of skincare concerns, rather than going for\n\nare of course talking about one of the biggest trends going around the skincare\n\nAnd really, in a world of overwhelming amount of skincare brands and products nowadays,\n\nexperts within the skincare realm, and getting to try loads of different\n\nlineup of luxury skincare brands.\n\nof skincare concerns where you're going to see really nice differences in your skin.\n\nreally, really love Make's approach to skincare, formulating skincare, and\n\nskincare products with you, but today we're going to concentrate on K-Beauty. You know I love K-Beauty.\n\nListen, anyone can launch a skincare brand.\n\nskincare 